In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from data_preprocess import DataPreprocessor
from sklearn.preprocessing import StandardScaler

In [47]:
airline_delay_df = pd.read_csv('data/flights_sample_3m.csv')
airline_codes_df = pd.read_csv('data/airline_codes.csv')
airline_fleet_df = pd.read_csv('data/Fleet Data.csv')
airline_prices_df = pd.read_csv('data/consumer_airfare.csv')
# busiestairports_df = pd.read_csv('data/busiestAirports.csv', encoding='cp1252')
# passenger_satisfaction_df = pd.read_excel('data/passenger_satisfaction.xlsx')
storm_events_df = pd.read_csv('data/StormEvents_details-ftp_v1.0_d2020_c20240620.csv')
international_report_departures_df = pd.read_csv('data/International_Report_Departures.csv')
# international_report_passengers_df = pd.read_csv('../Data/international_report_passengers.csv')

In [48]:
state_dict = {
    "ALABAMA": "AL",
    "ALASKA": "AK",
    "ARIZONA": "AZ",
    "ARKANSAS": "AR",
    "CALIFORNIA": "CA",
    "COLORADO": "CO",
    "CONNECTICUT": "CT",
    "DELAWARE": "DE",
    "DISTRICT OF COLUMBIA": "DC",
    "FLORIDA": "FL",
    "GEORGIA": "GA",
    "HAWAII": "HI",
    "IDAHO": "ID",
    "ILLINOIS": "IL",
    "INDIANA": "IN",
    "IOWA": "IA",
    "KANSAS": "KS",
    "KENTUCKY": "KY",
    "LOUISIANA": "LA",
    "MAINE": "ME",
    "MARYLAND": "MD",
    "MASSACHUSETTS": "MA",
    "MICHIGAN": "MI",
    "MINNESOTA": "MN",
    "MISSISSIPPI": "MS",
    "MISSOURI": "MO",
    "MONTANA": "MT",
    "NEBRASKA": "NE",
    "NEVADA": "NV",
    "NEW HAMPSHIRE": "NH",
    "NEW JERSEY": "NJ",
    "NEW MEXICO": "NM",
    "NEW YORK": "NY",
    "NORTH CAROLINA": "NC",
    "NORTH DAKOTA": "ND",
    "OHIO": "OH",
    "OKLAHOMA": "OK",
    "OREGON": "OR",
    "PENNSYLVANIA": "PA",
    "PUERTO RICO": "PR",
    "RHODE ISLAND": "RI",
    "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD",
    "TENNESSEE": "TN",
    "TEXAS": "TX",
    "UTAH": "UT",
    "VERMONT": "VT",
    "VIRGINIA": "VA",
    "VIRGIN ISLANDS": "VI",
    "WASHINGTON": "WA",
    "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI",
    "WYOMING": "WY"
}


In [49]:
# storm_events_df['year'] = storm_events_df['BEGIN_YEARMONTH'].astype(str).str[:4].astype(int)
# storm_events_df['month'] = storm_events_df['BEGIN_YEARMONTH'].astype(str).str[4:6].astype(int)
# storm_events_df['day'] = storm_events_df['BEGIN_DAY'].astype(int)
# storm_events_df['date'] = pd.to_datetime(storm_events_df[['year', 'month', 'day']])
# storm_events_df.drop(columns=['year', 'month', 'day'], inplace=True)

In [50]:
# international_report_departures_df['data_dte'] = international_report_departures_df['data_dte'].astype(str)
# international_report_departures_df['date'] = pd.to_datetime(international_report_departures_df['data_dte'])

international_report_departures_df['date'] = pd.to_datetime(
    international_report_departures_df['data_dte'], format="%Y-%m-%d"
)


In [51]:
storm_events_df['date'] = pd.to_datetime(storm_events_df['BEGIN_DATE_TIME'], format="%Y-%m-%d %H:%M:%S")

In [52]:
airline_delay_df['FL_DATE'] = pd.to_datetime(airline_delay_df['FL_DATE'], format='%Y-%m-%d')
airline_delay_df['year'] = airline_delay_df['FL_DATE'].dt.year
international_report_departures_df['year'] = international_report_departures_df['date'].dt.year

In [53]:
airline_delay_df_2020 = airline_delay_df[airline_delay_df['FL_DATE'].dt.year == 2020]
international_report_departures_df_2020 = international_report_departures_df[international_report_departures_df['date'].dt.year == 2020]

In [54]:
airline_delay_df_2020.columns

Index(['Unnamed: 0', 'FL_DATE', 'AIRLINE', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT', 'Month_Year', 'year'],
      dtype='object')

In [55]:
airline_delay_df_2020.fillna(0, inplace=True) 

C:\Users\muthu\AppData\Local\Temp\ipykernel_6864\2125564179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airline_delay_df_2020.fillna(0, inplace=True)


In [56]:
merged_df = airline_delay_df_2020.merge(
    international_report_departures_df_2020,
    left_on=['FL_DATE', 'AIRLINE_CODE'],   # List format for multiple columns
    right_on=['date', 'carrier'],         # List format for multiple columns
    how='left'
)


In [57]:
merged_df['state'] = merged_df['DEST_CITY'].str.split(", ").str[1]
storm_events_df['STATE'] = storm_events_df['STATE'].map(state_dict)

In [58]:
storm_events_df.columns

Index(['Unnamed: 0', 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'STATE', 'EVENT_TYPE',
       'MAGNITUDE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'date'],
      dtype='object')

In [59]:
merged_df = merged_df.merge(
    storm_events_df,
    left_on=['FL_DATE', 'state'],   # List format for multiple columns
    right_on=['date', 'STATE'],         # List format for multiple columns
    how='inner'
)


In [60]:
# Selecting relevant columns
needed_columns = [
'FL_DATE', 'AIRLINE_CODE', 'state','CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'DIVERTED', 'CRS_ELAPSED_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'
]


In [61]:
merged_df['state'].head(1)

0    TX
Name: state, dtype: object

In [62]:
# merged_df = merged_df[needed_columns]

In [63]:
# # Drop rows where essential numerical columns are missing
# airline_fleet_df.dropna(subset=['Current', 'Total', 'Average Age'], inplace=True)

# # Fill missing values in 'Orders' and 'Future' with 0
# airline_fleet_df[['Orders', 'Future']] = airline_fleet_df[['Orders', 'Future']].fillna(0)

# # Fill missing categorical values with "Unknown"
# airline_fleet_df[['Parent Airline', 'Airline', 'Aircraft Type']] = airline_fleet_df[
#     ['Parent Airline', 'Airline', 'Aircraft Type']
# ].fillna('Unknown')

# # Convert cost columns to numeric by removing '$' and ','
# airline_fleet_df['Unit Cost'] = (
#     airline_fleet_df['Unit Cost'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# )
# airline_fleet_df['Total Cost (Current)'] = (
#     airline_fleet_df['Total Cost (Current)'].replace({'\$': '', ',': ''}, regex=True).astype(float)
# )


In [64]:
airline_merged = airline_fleet_df.merge(
    airline_codes_df,
    left_on='Airline',   # List format for multiple columns
    right_on='Airline',         # List format for multiple columns
    how='left'
)


In [65]:
airline_merged = airline_merged.merge(
    airline_prices_df,
    left_on='IATA',   # List format for multiple columns
    right_on='car',         # List format for multiple columns
    how='inner'
)


In [66]:
airline_merged['state'] = airline_merged['city2'].str.split(", ").str[1]

In [67]:
airline_merged.columns

Index(['Unnamed: 0_x', 'Parent Airline', 'Airline', 'Aircraft Type', 'Current',
       'Historic', 'Unit Cost', 'Average Age', 'Unnamed: 0_y', 'IATA', 'ICAO',
       'Unnamed: 0', 'mkt_fare', 'citymarketid_1', 'citymarketid_2', 'city1',
       'city2', 'carairlineid', 'car', 'carpax', 'carpaxshare', 'Date',
       'state'],
      dtype='object')

MERGE BELOW AFTER CLEANING

In [68]:
from data_preprocess import DataPreprocessor

In [69]:
# # Selecting relevant columns
# needed_columns = [
# 'IATA', 'Total', 'Orders', 'Unit Cost', 'Total Cost (Current)', 'Average Age','state'
# ]

# airline_merged = airline_merged[needed_columns]


In [70]:
len(merged_df)

3355730

In [71]:
merged_df.columns

Index(['Unnamed: 0_x', 'FL_DATE', 'AIRLINE', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT', 'Month_Year', 'year_x',
       'Unnamed: 0_y', 'data_dte', 'Year', 'Month', 'usg_apt_id', 'usg_apt',
       'usg_wac', 'fg_apt_id', 'fg_apt', 'fg_wac', 'airlineid', 'carrier',
       'carriergroup', 'type', 'Scheduled', 'Charter', 'date_x', 'year_y',
       'state', 'Unnamed: 0', 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'STATE',
       'EVENT_TYPE', 'MAGNITUDE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'date_y'],
      dtype='object')

In [72]:
# # Selecting relevant columns
# needed_columns = [
# 'FL_DATE', 'AIRLINE_CODE', 'state', 'ARR_DELAY', 'CANCELLED',
#        'DIVERTED', 'CRS_ELAPSED_TIME', 'DISTANCE','INJURIES_DIRECT', 'INJURIES_INDIRECT',
#        'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
#        'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY'
# ]

# merged_df = merged_df[needed_columns]


In [73]:
final_df = DataPreprocessor(merged_df).handle_outliers()

d:\florida_coursework\second_sem\intro_to_data_science\Scripts\Scripts\data_preprocess.py:110: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = zscore(col_data)
d:\florida_coursework\second_sem\intro_to_data_science\Scripts\Scripts\data_preprocess.py:110: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = zscore(col_data)
d:\florida_coursework\second_sem\intro_to_data_science\Scripts\Scripts\data_preprocess.py:110: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = zscore(col_data)


Removing 1701967 rows flagged as outliers across numeric columns.


In [74]:
final_df.columns

Index(['Unnamed: 0_x', 'FL_DATE', 'AIRLINE', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT', 'Month_Year', 'year_x',
       'Unnamed: 0_y', 'data_dte', 'Year', 'Month', 'usg_apt_id', 'usg_apt',
       'usg_wac', 'fg_apt_id', 'fg_apt', 'fg_wac', 'airlineid', 'carrier',
       'carriergroup', 'type', 'Scheduled', 'Charter', 'date_x', 'year_y',
       'state', 'Unnamed: 0', 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'STATE',
       'EVENT_TYPE', 'MAGNITUDE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'date_y'],
      dtype='object')

In [75]:
# merged_df.to_csv('data/merged_df.csv', index=False)
# airline_merged.to_csv('data/airline_merged.csv', index=False)

In [76]:
null_counts = merged_df.isnull().sum()
print(null_counts)


Unnamed: 0_x                    0
FL_DATE                         0
AIRLINE                         0
AIRLINE_CODE                    0
DOT_CODE                        0
FL_NUMBER                       0
ORIGIN                          0
ORIGIN_CITY                     0
DEST                            0
DEST_CITY                       0
CRS_DEP_TIME                    0
DEP_DELAY                       0
CRS_ARR_TIME                    0
ARR_DELAY                       0
CANCELLED                       0
DIVERTED                        0
CRS_ELAPSED_TIME                0
ELAPSED_TIME                    0
AIR_TIME                        0
DISTANCE                        0
DELAY_DUE_CARRIER               0
DELAY_DUE_WEATHER               0
DELAY_DUE_NAS                   0
DELAY_DUE_SECURITY              0
DELAY_DUE_LATE_AIRCRAFT         0
Month_Year                      0
year_x                          0
Unnamed: 0_y               149531
data_dte                   149531
Year          

In [77]:
final_merged_df = merged_df.dropna(subset=['MAGNITUDE'])

In [78]:
final_merged_df.columns

Index(['Unnamed: 0_x', 'FL_DATE', 'AIRLINE', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'DIVERTED', 'CRS_ELAPSED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS',
       'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT', 'Month_Year', 'year_x',
       'Unnamed: 0_y', 'data_dte', 'Year', 'Month', 'usg_apt_id', 'usg_apt',
       'usg_wac', 'fg_apt_id', 'fg_apt', 'fg_wac', 'airlineid', 'carrier',
       'carriergroup', 'type', 'Scheduled', 'Charter', 'date_x', 'year_y',
       'state', 'Unnamed: 0', 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'STATE',
       'EVENT_TYPE', 'MAGNITUDE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'date_y'],
      dtype='object')

In [79]:
final_merged_df.drop_duplicates(subset=['FL_DATE','AIRLINE_CODE','state','MAGNITUDE','ARR_DELAY'], inplace=True)

In [80]:
final_merged_df.to_csv('data/final_merged_df.csv', index=False)